# Three Neurons

In [3]:
import numpy as np
import random
import math

## 1. Three Neurons Model

In [4]:
class Neuron1:
    def __init__(self):
        self.w1 = np.array([random.random(), random.random()])   # weight of one input
        self.b1 = np.array([random.random()])   # bias
        print("Neuron1 - Initial w1: {0}, b1: {1}".format(self.w1, self.b1))

    def u1(self, x):
        return np.dot(self.w1, x) + self.b1

    def f(self, u1):
        return max(0.0, u1)

    def z1(self, x):
        u1 = self.u1(x)
        return self.f(u1)

class Neuron2:
    def __init__(self):
        self.w2 = np.array([random.random(), random.random()])   # weight of one input
        self.b2 = np.array([random.random()])   # bias
        print("Neuron2 - Initial w2: {0}, b2: {1}".format(self.w2, self.b2))

    def u2(self, x):
        return np.dot(self.w2, x) + self.b2

    def f(self, u2):
        return max(0.0, u2)

    def z2(self, x):
        u2 = self.u2(x)
        return self.f(u2)

class Neuron3:
    def __init__(self, n1, n2):
        self.w3 = np.array([random.random(), random.random()])   # weight of one input
        self.b3 = np.array([random.random()])   # bias
        self.n1 = n1
        self.n2 = n2
        print("Neuron2 - Initial w3: {0}, b3: {1}".format(self.w3, self.b3))

    def u3(self, x):
        z1 = self.n1.z1(x)
        z2 = self.n2.z2(x)
        z = np.array([z1, z2])
        return np.dot(self.w3, z) + self.b3

    def f(self, u3):
        return max(0.0, u3)

    def z3(self, x):
        u3 = self.u3(x)
        return self.f(u3)

    def squared_error(self, x, z_target):
        return 1.0 / 2.0 * math.pow(self.z3(x) - z_target, 2)

    def numerical_derivative(self, params, x, z_target):
        delta = 1e-4 # 0.0001
        grad = np.zeros_like(params)
        
        for idx in range(params.size):
            temp_val = params[idx]

            #f(x + delta) 계산
            params[idx] = params[idx] + delta
            fxh1 = self.squared_error(x, z_target)
            
            #f(x - delta) 계산
            params[idx] = params[idx] - delta
            fxh2 = self.squared_error(x, z_target)
            
            #f(x + delta) - f(x - delta) / 2 * delta 계산
            grad[idx] = (fxh1 - fxh2) / (2 * delta)
            params[idx] = temp_val
        return grad

    def learning(self, alpha, maxEpoch, data):
        print_epoch_period = 50
        for i in range(maxEpoch):
            for idx in range(data.numTrainData):
                x = data.training_input_value[idx]
                z_target = data.training_z_target[idx]

                self.n1.w1 = self.n1.w1 - alpha * self.numerical_derivative(self.n1.w1, x, z_target)
                self.n1.b1 = self.n1.b1 - alpha * self.numerical_derivative(self.n1.b1, x, z_target)
                self.n2.w2 = self.n2.w2 - alpha * self.numerical_derivative(self.n2.w2, x, z_target)
                self.n2.b2 = self.n2.b2 - alpha * self.numerical_derivative(self.n2.b2, x, z_target)
                self.w3 = self.w3 - alpha * self.numerical_derivative(self.w3, x, z_target)
                self.b3 = self.b3 - alpha * self.numerical_derivative(self.b3, x, z_target)

            if i % print_epoch_period == 0:
                sum = 0.0
                for idx in range(data.numTrainData):
                    sum = sum + self.squared_error(data.training_input_value[idx], data.training_z_target[idx])
                print("Epoch {0:3d}: Err: {1:5.3f}, w1_0: {2:5.3f}, w1_1: {3:5.3f}, b1: {4:5.3f}, w2_0: {5:5.3f}, w2_1: {6:5.3f}, b2: {7:5.3f}, w3_0: {8:5.3f}, w3_1: {9:5.3f}, b3: {10:5.3f}".format(
                    i, 
                    sum / data.numTrainData,
                    self.n1.w1[0],
                    self.n1.w1[1],
                    self.n1.b1[0],
                    self.n2.w2[0],
                    self.n2.w2[1],
                    self.n2.b2[0],                      
                    self.w3[0],
                    self.w3[1],
                    self.b3[0])
                )

## 2. OR Gate with Three Neurons

In [5]:
class Data:
    def __init__(self):
        self.training_input_value = np.array([(0.0, 0.0), (1.0, 0.0), (0.0, 1.0), (1.0, 1.0)])
        self.training_z_target = np.array([0.0, 1.0, 1.0, 1.0])
        self.numTrainData = len(self.training_input_value)

if __name__ == '__main__':
    n1 = Neuron1()
    n2 = Neuron2()
    n3 = Neuron3(n1, n2)
    d = Data()
    for idx in range(d.numTrainData):
        x = d.training_input_value[idx]
        z3 = n3.z3(x)
        z_target = d.training_z_target[idx]
        error = n3.squared_error(x, z_target)
        print("x: {0:s}, z3: {1:s}, z_target: {2:s}, error: {3:7.5f}".format(str(x), str(z3), str(z_target), error))        

    n3.learning(0.05, 1000, d)

    for idx in range(d.numTrainData):
        x = d.training_input_value[idx]
        z3 = n3.z3(x)
        z_target = d.training_z_target[idx]
        error = n3.squared_error(x, z_target)
        print("x: {0:s}, z3: {1:s}, z_target: {2:s}, error: {3:7.5f}".format(str(x), str(z3), str(z_target), error))        

Neuron1 - Initial w1: [ 0.38522756  0.20353551], b1: [ 0.42084058]
Neuron2 - Initial w2: [ 0.1493424   0.21953654], b2: [ 0.8024181]
Neuron2 - Initial w3: [ 0.796124    0.13468804], b3: [ 0.09694243]
x: [ 0.  0.], z3: [ 0.54005983], z_target: 0.0, error: 0.14583
x: [ 1.  0.], z3: [ 0.86686338], z_target: 1.0, error: 0.00886
x: [ 0.  1.], z3: [ 0.73166828], z_target: 1.0, error: 0.03600
x: [ 1.  1.], z3: [ 1.05847183], z_target: 1.0, error: 0.00171
Epoch   0: Err: 0.048, w1_0: 0.387, w1_1: 0.208, b1: 0.418, w2_0: 0.150, w2_1: 0.220, b2: 0.802, w3_0: 0.797, w3_1: 0.133, b3: 0.093
Epoch  50: Err: 0.035, w1_0: 0.468, w1_1: 0.376, b1: 0.333, w2_0: 0.160, w2_1: 0.243, b2: 0.785, w3_0: 0.867, w3_1: 0.118, b3: -0.008
Epoch 100: Err: 0.032, w1_0: 0.495, w1_1: 0.442, b1: 0.292, w2_0: 0.161, w2_1: 0.251, b2: 0.777, w3_0: 0.910, w3_1: 0.110, b3: -0.050
Epoch 150: Err: 0.032, w1_0: 0.500, w1_1: 0.461, b1: 0.275, w2_0: 0.159, w2_1: 0.252, b2: 0.772, w3_0: 0.933, w3_1: 0.108, b3: -0.064
Epoch 200: Er

## 3. AND Gate with Three Neurons

In [7]:
class Data:
    def __init__(self):
        self.training_input_value = np.array([(0.0, 0.0), (1.0, 0.0), (0.0, 1.0), (1.0, 1.0)])
        self.training_z_target = np.array([0.0, 0.0, 0.0, 1.0])
        self.numTrainData = len(self.training_input_value)

if __name__ == '__main__':
    n1 = Neuron1()
    n2 = Neuron2()
    n3 = Neuron3(n1, n2)
    d = Data()
    for idx in range(d.numTrainData):
        x = d.training_input_value[idx]
        z3 = n3.z3(x)
        z_target = d.training_z_target[idx]
        error = n3.squared_error(x, z_target)
        print("x: {0:s}, z3: {1:s}, z_target: {2:s}, error: {3:7.5f}".format(str(x), str(z3), str(z_target), error))        

    n3.learning(0.05, 1000, d)

    for idx in range(d.numTrainData):
        x = d.training_input_value[idx]
        z3 = n3.z3(x)
        z_target = d.training_z_target[idx]
        error = n3.squared_error(x, z_target)
        print("x: {0:s}, z3: {1:s}, z_target: {2:s}, error: {3:7.5f}".format(str(x), str(z3), str(z_target), error))        

Neuron1 - Initial w1: [ 0.20432704  0.64053379], b1: [ 0.38208965]
Neuron2 - Initial w2: [ 0.927959    0.46040146], b2: [ 0.87461856]
Neuron2 - Initial w3: [ 0.73126551  0.1859103 ], b3: [ 0.88054529]
x: [ 0.  0.], z3: [ 1.32255487], z_target: 0.0, error: 0.87458
x: [ 1.  0.], z3: [ 1.64448932], z_target: 0.0, error: 1.35217
x: [ 0.  1.], z3: [ 1.87654852], z_target: 0.0, error: 1.76072
x: [ 1.  1.], z3: [ 2.19848297], z_target: 1.0, error: 0.71818
Epoch   0: Err: 0.518, w1_0: 0.167, w1_1: 0.602, b1: 0.293, w2_0: 0.922, w2_1: 0.456, b2: 0.859, w3_0: 0.649, w3_1: 0.007, b3: 0.766
Epoch  50: Err: 0.047, w1_0: 0.411, w1_1: 0.639, b1: -0.036, w2_0: 0.821, w2_1: 0.427, b2: 0.938, w3_0: 0.738, w3_1: -0.142, b3: 0.182
Epoch 100: Err: 0.021, w1_0: 0.611, w1_1: 0.689, b1: -0.187, w2_0: 0.784, w2_1: 0.415, b2: 0.955, w3_0: 0.919, w3_1: -0.111, b3: -0.030
Epoch 150: Err: 0.007, w1_0: 0.742, w1_1: 0.746, b1: -0.306, w2_0: 0.767, w2_1: 0.407, b2: 0.966, w3_0: 1.067, w3_1: -0.111, b3: -0.163
Epoch 2

## 4. XOR Gate with Three Neurons

In [12]:
class Data:
    def __init__(self):
        self.training_input_value = np.array([(0.0, 0.0), (1.0, 0.0), (0.0, 1.0), (1.0, 1.0)])
        self.training_z_target = np.array([0.0, 1.0, 1.0, 0.0])
        self.numTrainData = len(self.training_input_value)

if __name__ == '__main__':
    n1 = Neuron1()
    n2 = Neuron2()
    n3 = Neuron3(n1, n2)
    d = Data()
    for idx in range(d.numTrainData):
        x = d.training_input_value[idx]
        z3 = n3.z3(x)
        z_target = d.training_z_target[idx]
        error = n3.squared_error(x, z_target)
        print("x: {0:s}, z3: {1:s}, z_target: {2:s}, error: {3:7.5f}".format(str(x), str(z3), str(z_target), error))        

    n3.learning(0.05, 1000, d)

    for idx in range(d.numTrainData):
        x = d.training_input_value[idx]
        z3 = n3.z3(x)
        z_target = d.training_z_target[idx]
        error = n3.squared_error(x, z_target)
        print("x: {0:s}, z3: {1:s}, z_target: {2:s}, error: {3:7.5f}".format(str(x), str(z3), str(z_target), error))        

Neuron1 - Initial w1: [ 0.98041965  0.33318097], b1: [ 0.90704043]
Neuron2 - Initial w2: [ 0.24376458  0.0993518 ], b2: [ 0.14829085]
Neuron2 - Initial w3: [ 0.73292247  0.86866256], b3: [ 0.31973756]
x: [ 0.  0.], z3: [ 1.11334259], z_target: 0.0, error: 0.61977
x: [ 1.  0.], z3: [ 2.04366335], z_target: 1.0, error: 0.54462
x: [ 0.  1.], z3: [ 1.4438416], z_target: 1.0, error: 0.09850
x: [ 1.  1.], z3: [ 2.37416236], z_target: 0.0, error: 2.81832
Epoch   0: Err: 0.426, w1_0: 0.930, w1_1: 0.295, b1: 0.833, w2_0: 0.181, w2_1: 0.051, b2: 0.059, w3_0: 0.565, w3_1: 0.843, b3: 0.226
Epoch  50: Err: 0.110, w1_0: 0.789, w1_1: 0.242, b1: 0.766, w2_0: 0.221, w2_1: -0.278, b2: 0.041, w3_0: 0.169, w3_1: 0.895, b3: 0.187
Epoch 100: Err: 0.092, w1_0: 0.742, w1_1: 0.252, b1: 0.765, w2_0: 0.413, w2_1: -0.421, b2: 0.000, w3_0: 0.067, w3_1: 1.004, b3: 0.289
Epoch 150: Err: 0.084, w1_0: 0.727, w1_1: 0.248, b1: 0.757, w2_0: 0.522, w2_1: -0.529, b2: 0.002, w3_0: -0.026, w3_1: 1.096, b3: 0.374
Epoch 200: E